<a href="https://colab.research.google.com/github/sejalxz/react/blob/main/addition_of_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [5]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-fwsv5lkn
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-fwsv5lkn
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=dfd3e3f098038a13776839622c326e04c0591ce65a1aa087c412d3f57ff65932
  Stored in directory: /tmp/pip-ephem-wheel-cache-uf0twuck/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include <stdio.h>
#include <cuda.h>

#define SIZE 100

__global__ void vectorAdd(int *a, int *b, int *c, int size)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size)
    {
        c[tid] = a[tid] + b[tid];
    }
}

int main()
{
    int a[SIZE], b[SIZE], c[SIZE];
    int size = SIZE;

    // Initialize input vectors
    for (int i = 0; i < size; i++)
    {
        a[i] = i*10;
        b[i] = i;
    }

    int *dev_a, *dev_b, *dev_c;
    cudaMalloc((void **)&dev_a, size * sizeof(int));
    cudaMalloc((void **)&dev_b, size * sizeof(int));
    cudaMalloc((void **)&dev_c, size * sizeof(int));

    cudaMemcpy(dev_a, a, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size * sizeof(int), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;

    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(dev_a, dev_b, dev_c, size);

    cudaMemcpy(c, dev_c, size * sizeof(int), cudaMemcpyDeviceToHost);

    // Print the result
    printf("Vector Addition (a, b, c):\n");
    for (int i = 0; i < size; i++)
    {
        printf("%d + %d = %d\n", a[i], b[i], c[i]);
    }

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}


Vector Addition (a, b, c):
0 + 0 = 0
10 + 1 = 11
20 + 2 = 22
30 + 3 = 33
40 + 4 = 44
50 + 5 = 55
60 + 6 = 66
70 + 7 = 77
80 + 8 = 88
90 + 9 = 99
100 + 10 = 110
110 + 11 = 121
120 + 12 = 132
130 + 13 = 143
140 + 14 = 154
150 + 15 = 165
160 + 16 = 176
170 + 17 = 187
180 + 18 = 198
190 + 19 = 209
200 + 20 = 220
210 + 21 = 231
220 + 22 = 242
230 + 23 = 253
240 + 24 = 264
250 + 25 = 275
260 + 26 = 286
270 + 27 = 297
280 + 28 = 308
290 + 29 = 319
300 + 30 = 330
310 + 31 = 341
320 + 32 = 352
330 + 33 = 363
340 + 34 = 374
350 + 35 = 385
360 + 36 = 396
370 + 37 = 407
380 + 38 = 418
390 + 39 = 429
400 + 40 = 440
410 + 41 = 451
420 + 42 = 462
430 + 43 = 473
440 + 44 = 484
450 + 45 = 495
460 + 46 = 506
470 + 47 = 517
480 + 48 = 528
490 + 49 = 539
500 + 50 = 550
510 + 51 = 561
520 + 52 = 572
530 + 53 = 583
540 + 54 = 594
550 + 55 = 605
560 + 56 = 616
570 + 57 = 627
580 + 58 = 638
590 + 59 = 649
600 + 60 = 660
610 + 61 = 671
620 + 62 = 682
630 + 63 = 693
640 + 64 = 704
650 + 65 = 715
660 + 66 = 726


In [9]:
%%cu
#include <stdio.h>
#include <cuda.h>
#include <chrono>

#define SIZE 500

__global__ void vectorAdd(int *a, int *b, int *c, int size)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size)
    {
        c[tid] = a[tid] + b[tid];
    }
}

void vectorAddCPU(int *a, int *b, int *c, int size)
{
    for (int i = 0; i < size; i++)
    {
        c[i] = a[i] + b[i];
    }
}

int main()
{
    int a[SIZE], b[SIZE], c[SIZE];
    int size = SIZE;

    // Initialize input vectors
    for (int i = 0; i < size; i++)
    {
        a[i] = i * 10;
        b[i] = i;
    }

    int *dev_a, *dev_b, *dev_c;
    cudaMalloc((void **)&dev_a, size * sizeof(int));
    cudaMalloc((void **)&dev_b, size * sizeof(int));
    cudaMalloc((void **)&dev_c, size * sizeof(int));

    cudaMemcpy(dev_a, a, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size * sizeof(int), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;

    // Measure GPU time using CUDA events
    cudaEvent_t startGPU, stopGPU;
    cudaEventCreate(&startGPU);
    cudaEventCreate(&stopGPU);

    cudaEventRecord(startGPU);
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(dev_a, dev_b, dev_c, size);
    cudaEventRecord(stopGPU);
    cudaEventSynchronize(stopGPU);

    float gpuMilliseconds = 0;
    cudaEventElapsedTime(&gpuMilliseconds, startGPU, stopGPU);
    float gpuTime = gpuMilliseconds / 1000.0;

    cudaMemcpy(c, dev_c, size * sizeof(int), cudaMemcpyDeviceToHost);

    // Measure CPU time using std::chrono
    auto cpuStartTime = std::chrono::high_resolution_clock::now();
    vectorAddCPU(a, b, c, size);
    auto cpuEndTime = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> cpuDuration = cpuEndTime - cpuStartTime;
    float cpuTime = cpuDuration.count();

    // Print the result
    printf("Vector Addition (a, b, c):\n");
    for (int i = 0; i < size; i++)
    {
        printf("%d + %d = %d\n", a[i], b[i], c[i]);
    }

    // Calculate speedup
    float speedup = cpuTime / gpuTime;

    // Print CPU, GPU time, and speedup
    printf("CPU Time: %.6f seconds\n", cpuTime);
    printf("GPU Time: %.6f seconds\n", gpuTime);
    printf("Speedup: %.2f\n", speedup);

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}


Vector Addition (a, b, c):
0 + 0 = 0
10 + 1 = 11
20 + 2 = 22
30 + 3 = 33
40 + 4 = 44
50 + 5 = 55
60 + 6 = 66
70 + 7 = 77
80 + 8 = 88
90 + 9 = 99
100 + 10 = 110
110 + 11 = 121
120 + 12 = 132
130 + 13 = 143
140 + 14 = 154
150 + 15 = 165
160 + 16 = 176
170 + 17 = 187
180 + 18 = 198
190 + 19 = 209
200 + 20 = 220
210 + 21 = 231
220 + 22 = 242
230 + 23 = 253
240 + 24 = 264
250 + 25 = 275
260 + 26 = 286
270 + 27 = 297
280 + 28 = 308
290 + 29 = 319
300 + 30 = 330
310 + 31 = 341
320 + 32 = 352
330 + 33 = 363
340 + 34 = 374
350 + 35 = 385
360 + 36 = 396
370 + 37 = 407
380 + 38 = 418
390 + 39 = 429
400 + 40 = 440
410 + 41 = 451
420 + 42 = 462
430 + 43 = 473
440 + 44 = 484
450 + 45 = 495
460 + 46 = 506
470 + 47 = 517
480 + 48 = 528
490 + 49 = 539
500 + 50 = 550
510 + 51 = 561
520 + 52 = 572
530 + 53 = 583
540 + 54 = 594
550 + 55 = 605
560 + 56 = 616
570 + 57 = 627
580 + 58 = 638
590 + 59 = 649
600 + 60 = 660
610 + 61 = 671
620 + 62 = 682
630 + 63 = 693
640 + 64 = 704
650 + 65 = 715
660 + 66 = 726
